In [5]:
import bilby
import tbilby
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from bilby.core.result import read_in_result
import pandas as pd
from corner import corner
import matplotlib.lines as mlines

In [6]:
x=np.linspace(0,150,1501)
mu=np.array([35, 74, 101])

model=[]
mock_data=[]
sigma=[10,8,12]
A=[1.0,0.8,1.2]
# sigma=[10,10,10]
# A=[1.0,1.0,1.0]
for i in range(3):
    model.append(norm(mu[i],sigma[i]))
    mock_data.append(A[i]*model[i].pdf(x))
    
noise_model=norm(loc=0,scale=0.15)
np.random.seed(1234)
mock_noise=noise_model.rvs(1501)

data=mock_data[0]+mock_data[1]+mock_data[2]+mock_noise

In [7]:
n_peaks=6

In [8]:
def gauss_pulse(time, A, mu, sigma):
    return A * norm(mu,sigma).pdf(time)

In [9]:
component_functions_dict={}
component_functions_dict[gauss_pulse]=(n_peaks,'mu', 'A', 'sigma')

In [10]:
signal_model = tbilby.core.base.create_transdimensional_model('model',  component_functions_dict,\
                                                       returns_polarization=False,SaveTofile=False)

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'np', 'pd', 'infer_parameters_from_function', 'bilby', 'DiscreteUniform', 'create_transdimensional_priors', '_create_priors_with_nested_condition_functions', 'create_transdimensional_model', 'create_plain_priors', 'extract_maximal_likelihood_param_values', '_fix_posterior_if_needed', 'preprocess_results', '_recluster_one_dim_posterior_experimental_use_it_wisely', '_group_params_by_numerical_ending', 'gauss_pulse'])


In [11]:
# Now lets instantiate a version of our GaussianLikelihood, giving it
# the time, data and signal model
# likelihood = bilby.likelihood.GaussianLikelihood(time, data, model, sigma)
time=np.linspace(0,150,1501)
likelihood = bilby.likelihood.GaussianLikelihood(time, data, signal_model, sigma=0.15*np.ones(len(time)))

In [12]:
# to define Transdimensional Conditional prior you need to inherit from the relevant prior
class TransdimensionalConditionalUniform(tbilby.core.prior.TransdimensionalConditionalUniform):
   # one must define the transdimensional_condition_function function, so we know what to do with conditional variables...  
    # it is an abstract function, without it you cant instantiate this class 
    def transdimensional_condition_function(self,**required_variables):
        ''' setting the mimmum according the the last peak value of the gaussian.
        Here you refer to the parameters you are 
        working with '''
        # mu is returned as an array 
        
        # minimum = self.minimum
        # if(len(self.mu)>0): # handle the first mu case
        #     minimum = self.mu[-1]               
        #     setattr(self,'minimum',minimum)  # setting the atribute of the class
        # return dict(minimum=minimum)
        minimum = self.minimum
        if(len(self.mu)>0): # handle the first mu case
            minimum = self.mu[-1]
            above = minimum > self.maximum
            try:
                minimum[above] = self.minimum[0]
            except:
                minimum[above] = self.minimum
        setattr(self,'minimum',minimum)  # setting the atribute of the class
            
        return dict(minimum=minimum)

In [13]:
# prameter_prior
priors = bilby.core.prior.dict.ConditionalPriorDict()
priors =tbilby.core.base.create_transdimensional_priors(transdimensional_prior_class=TransdimensionalConditionalUniform,\
                                                          param_name='mu',\
                                                          nmax= n_peaks,\
                                                          nested_conditional_transdimensional_params=['mu'],\
                                                          conditional_transdimensional_params=[],\
                                                          conditional_params=[],\
                                                          prior_dict_to_add=priors,\
                                                          SaveConditionFunctionsToFile=False,\
                                                          minimum= 0,maximum=150)
priors["A0"] = bilby.core.prior.Uniform(0.5, 1.5, "A0")
priors["A1"] = bilby.core.prior.Uniform(0.5, 1.5, "A1")
priors["A2"] = bilby.core.prior.Uniform(0.5, 1.5, "A2")
priors["A3"] = bilby.core.prior.Uniform(0.5, 1.5, "A3")
priors["A4"] = bilby.core.prior.Uniform(0.5, 1.5, "A4")
priors["A5"] = bilby.core.prior.Uniform(0.5, 1.5, "A5")
priors["sigma0"] = bilby.core.prior.Uniform(5, 20, "sigma0")
priors["sigma1"] = bilby.core.prior.Uniform(5, 20, "sigma1")
priors["sigma2"] = bilby.core.prior.Uniform(5, 20, "sigma2")
priors["sigma3"] = bilby.core.prior.Uniform(5, 20, "sigma3")
priors["sigma4"] = bilby.core.prior.Uniform(5, 20, "sigma4")
priors["sigma5"] = bilby.core.prior.Uniform(5, 20, "sigma5")

priors['n_gauss_pulse'] = tbilby.core.prior.DiscreteUniform(0,n_peaks,'n_gauss_pulse')

In [15]:
result = bilby.core.sampler.run_sampler(
    likelihood,
    priors=priors,
    sampler="dynesty",
    outdir='dynesty_full_more',
    label='dynesty',
    nlive= 1000,
    resume=True,
    npool=1,
)